# Finetuning using IndoBART

In [ ]:
! git clone https://github.com/indobenchmark/indonlg.git

Cloning into 'indonlg'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 152 (delta 75), reused 97 (delta 34), pack-reused 0
Receiving objects: 100% (152/152), 2.32 MiB | 10.90 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
%cd /content/indonlg
! pip install -r requirements.txt

/content/indonlg
     |████████████████████████████████| 1.5 MB 6.7 MB/s 
     |████████████████████████████████| 68 kB 1.6 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 186 kB 71.2 MB/s 
     |████████████████████████████████| 3.3 MB 60.7 MB/s 
     |████████████████████████████████| 394.7 MB 18 kB/s 
     |████████████████████████████████| 1.2 MB 56.1 MB/s 
     |████████████████████████████████| 22.3 MB 959 kB/s 
     |████████████████████████████████| 65 kB 5.0 MB/s 
     |████████████████████████████████| 149 kB 73.5 MB/s 
     |████████████████████████████████| 1.8 MB 56.7 MB/s 
     |████████████████████████████████| 212 kB 77.3 MB/s 
     |████████████████████████████████| 136 kB 74.3 MB/s 
     |████████████████████████████████| 132 kB 72.1 MB/s 
     |████████████████████████████████| 3.8 MB 55.7 MB/s 
     |████████████████████████████████| 2.9 MB 53.8 MB/s 
     |████████████████████████████████| 462 kB 58.4 MB/s 
  

In [ ]:
%cd /content/indonlg/examples

/content/indonlg/examples


In [ ]:
import os, sys
sys.path.append('../')
os.chdir('../')

import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import optim
from transformers import MBartForConditionalGeneration

from indobenchmark import IndoNLGTokenizer
from utils.train_eval import train, evaluate
from utils.metrics import generation_metrics_fn
from utils.forward_fn import forward_generation
from utils.data_utils import MachineTranslationDataset, GenerationDataLoader

In [ ]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
# Set random seed
# set_seed(26092020)

# Load Model

In [ ]:
bart_model = MBartForConditionalGeneration.from_pretrained('indobenchmark/indobart-v2')
tokenizer = IndoNLGTokenizer.from_pretrained('indobenchmark/indobart-v2')

model = bart_model
model

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [ ]:
count_param(model)

131543040

# Prepare Dataset

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = './save/MT_SUNIBS_INZNTV/example_jv_id'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_path = './dataset/MT_JAVNRF_INZNTV/train_preprocess.json'
valid_dataset_path = './dataset/MT_JAVNRF_INZNTV/valid_preprocess.json'
test_dataset_path = './dataset/MT_JAVNRF_INZNTV/test_preprocess.json'

train_dataset = MachineTranslationDataset(train_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset = MachineTranslationDataset(valid_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset = MachineTranslationDataset(test_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

# Test model to generate sequences

In [ ]:
inputs = ['aku pergi ke toko obat membeli <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[indonesian]', decoder_lang_token='[indonesian]')

bart_input.to(device)
bart_out = model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku pergi ke toko obat membeli <mask> [indonesian]
<s> aku pergi ke toko obat membeli obat.


In [ ]:
inputs = ['aku menyang pasar karo <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[javanese]', decoder_lang_token='[javanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku menyang pasar karo <mask> [javanese]
<s> aku menyang pasar karo tuku </s>


In [ ]:
inputs = ['kuring ka pasar senen meuli daging <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[sundanese]', decoder_lang_token='[sundanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> kuring ka pasar senen meuli daging <mask> [sundanese]
<s> kuring ka pasar senen meuli daging sapi.


# Test model to translate

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [03:25<00:00,  1.37s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result.csv")

== Prediction Result ==
                                                 hyp  \
0  samana h ér od <0xC3> <0xA8> s, raja - w ilaya...   
1  ing salebeting kawontenan ingkang makaten puni...   
2  saking punika panjenenganipun uninga sad <0xC3...   
3  - rah mat nung gila ing kowé kab <0xC3> <0xA8> h.   
4  18 : 6 - 7 aku nindak aké pagawé an kang abot....   

                                               label  
0  pada masa itu sampailah berita - berita tentan...  
1  " dan dalam keadaan demikian, ketika aku denga...  
2  karena itu ia telah melihat ke depan dan telah...  
3             kasih karunia menyertai kamu sekalian.  
4  kami melakukan pekerjaan tangan yang berat. ka...  

== Model Performance ==
           BLEU  SacreBLEU    ROUGE1    ROUGE2    ROUGEL  ROUGELsum
count  1.000000   1.000000  1.000000  1.000000  1.000000   1.000000
mean   1.475502   1.577628  7.544361  1.199374  7.336777   7.341366
std         NaN        NaN       NaN       NaN       NaN        NaN
min    1.475

In [ ]:
metric_df

,BLEU,SacreBLEU,ROUGE1,ROUGE2,ROUGEL,ROUGELsum
0,1.475502,1.577628,7.544361,1.199374,7.336777,7.341366


# Fine Tuning & Evaluation

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.1981 LR:0.00010000: 100%|██████████| 746/746 [01:38<00:00,  7.60it/s]


(Epoch 1) TRAIN LOSS:2.1981 BLEU:38.62 SacreBLEU:40.82 ROUGE1:58.77 ROUGE2:31.91 ROUGEL:55.51 ROUGELsum:55.51 LR:0.00010000


VALID LOSS:1.7435: 100%|██████████| 100/100 [00:04<00:00, 24.29it/s]


(Epoch 1) VALID LOSS:1.7435 BLEU:33.85 SacreBLEU:34.19 ROUGE1:62.11 ROUGE2:36.45 ROUGEL:59.85 ROUGELsum:59.85


(Epoch 2) TRAIN LOSS:1.3002 LR:0.00009000: 100%|██████████| 746/746 [01:37<00:00,  7.65it/s]


...

(Epoch 9) VALID LOSS:1.8983 BLEU:41.04 SacreBLEU:41.31 ROUGE1:67.57 ROUGE2:44.50 ROUGEL:65.83 ROUGELsum:65.85


(Epoch 10) TRAIN LOSS:0.0722 LR:0.00003874: 100%|██████████| 746/746 [01:38<00:00,  7.61it/s]


(Epoch 10) TRAIN LOSS:0.0722 BLEU:96.54 SacreBLEU:96.67 ROUGE1:98.40 ROUGE2:96.89 ROUGEL:98.39 ROUGELsum:98.39 LR:0.00003874


VALID LOSS:1.9346: 100%|██████████| 100/100 [00:04<00:00, 23.82it/s]


(Epoch 10) VALID LOSS:1.9346 BLEU:41.25 SacreBLEU:41.51 ROUGE1:67.70 ROUGE2:44.51 ROUGEL:65.76 ROUGELsum:65.76


In [ ]:
# Load best model
model.load_state_dict(torch.load(model_dir + "/best_model_0.th"))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [01:57<00:00,  1.28it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_jv-id.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_jv-id.csv")

== Prediction Result ==
                                                 hyp  \
0  pada waktu itu her odes, raja wilayah, mendeng...   
1  " dan dalam hal - hal yang demikian, ketika ak...   
2  karena itu ia mengetahui, sebelum dan mengatak...   
3                      kasih karunia menyertai kamu.   
4  dengan tanganku sendiri aku melakukan pekerjaa...   

                                               label  
0  pada masa itu sampailah berita - berita tentan...  
1  " dan dalam keadaan demikian, ketika aku denga...  
2  karena itu ia telah melihat ke depan dan telah...  
3             kasih karunia menyertai kamu sekalian.  
4  kami melakukan pekerjaan tangan yang berat. ka...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   38.709192  38.958457  64.439565  42.331416  61.407431  61.412404
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_dir+"/best_model_0.th"

'./save/MT_SUNIBS_INZNTV/example_jv_id/best_model_0.th'

In [ ]:
! cp './save/MT_SUNIBS_INZNTV/example_jv_id/best_model_0.th' '/content/drive/My Drive/'